<h1>Model Inference with RAG</h1>

## Setup

Change that following variable settings match your deployed model's *Inference endpoint*. for example: 

```
infer_endpoint = "https://model-vllm.apps.clusterx.sandboxx.opentlc.com"
```

In [1]:
# from file 3_RAG_with_Elastic
# client = Elasticsearch(['https://elasticsearch-sample-elasticsearch.apps.rosa-t59w8.oufo.p1.openshiftapps.com'], basic_auth=('elastic', '1rdzVXU8i8F9T8hs1p6c78d6'), verify_certs=False)


<h1>Model Inference</h1>

<h3>using: /v1/completions endpoint</h3>
<h4>*More for Single-shot prompts, single-turn Q&A or document completion</h4>

In [2]:
infer_endpoint = "http://model-predictor.minio.svc.cluster.local:8080" #Change infer endpoint here
infer_url = f"{infer_endpoint}/v1/completions"

In [3]:
# get the model name 
# for testing only
!curl http://model-predictor.minio.svc.cluster.local:8080/v1/models

{"object":"list","data":[{"id":"model","object":"model","created":1745399297,"owned_by":"vllm","root":"/mnt/models","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-6efda1d1ab644349b109915e5b0913d7","object":"model_permission","created":1745399297,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [4]:
import requests
# Test for hyperparameter tuning 
payload = {
    "model": "model",
    "prompt": "What's the difference between OpenShift and Kubernetes?",
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['text']
print(generated_response.strip())

OpenShift is a container application platform built around a core of Docker container packaging and Kubernetes orchestration. It provides an enterprise-grade container platform with originating technology contributions from Red Hat, Google, and others in the open source community.

Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications. It groups containers that make up an application into logical units for easy management and discovery.

In summary, OpenShift is a distribution of Kubernetes with additional features and tools tailored for enterprise use cases, while Kubernetes is the underlying orchestration engine for managing containers at scale.


In [5]:
# Test for hyperparameter tuning 
payload = {
    "model": "model",
    "prompt": "What's the difference between OpenShift and Kubernetes?",
    "max_tokens": 200,
    "temperature": 0.7
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['text']
print(generated_response.strip())

OpenShift is an open-source container application platform based on Kubernetes. It provides a more user-friendly and feature-rich environment for developers and operations teams, including built-in CI/CD pipelines, a web-based console, and integrated monitoring tools. Kubernetes, on the other hand, is a container orchestration system that automates the deployment, scaling, and management of containerized applications. It requires more manual configuration and integration with other tools.


<h1>Model Inference</h1>

<h3>using: /v1/chat/completions endpoint</h3>
<h3>*More for chat-tuned, multi-turn conversations, model behaving like assistant</h3>

In [6]:
##TO DO into the Prompt Template
# preserve messages history 
# keep older messages if they are essential for content 
# truncate long histories by summarizing/dropping older exchanges 

# Prompt token - is there a max? 

# create a Python class to connect to FE 

In [24]:
infer_endpoint = "http://model-predictor.minio.svc.cluster.local:8080" #Change infer endpoint here
infer_url = f"{infer_endpoint}/v1/chat/completions"

In [8]:
!pwd

/opt/app-root/src/SEAK_AI


In [9]:
# get the model name 
# for testing only
!curl http://model-predictor.minio.svc.cluster.local:8080/v1/models

{"object":"list","data":[{"id":"model","object":"model","created":1745399306,"owned_by":"vllm","root":"/mnt/models","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-85d97a29967141ffb8e2faef4ecfcde3","object":"model_permission","created":1745399306,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [10]:
user_question_one = "What's the difference between OpenShift & Kubernetes?"

messages = [
    {"role": "system", "content": "You are a chat bot assistant for helping people with their queries."},
    {"role": "user", "content": user_question_one}
]


payload = {
    "model": "model",
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

# add to the messages list 
messages.append({"role": "system", "content": generated_response.strip()})

OpenShift and Kubernetes are both container orchestration tools, but they have some key differences:

1. **Base Technology**: Kubernetes is an open-source platform designed to automate deploying, scaling, and managing containerized applications. It groups containers that make up an application into logical units for easy management and discovery. On the other hand, OpenShift is a container application platform built on top of Kubernetes. It adds additional features and capabilities.

2. **Ease of Use**: OpenShift is generally considered easier to use than raw Kubernetes because it includes a web console and automation features out of the box. This can be particularly beneficial for developers who may not have extensive experience with Kubernetes. 

3. **Additional Features**: OpenShift comes with several built-in features like integrated CI/CD pipelines (using tools such as Jenkins), a private registry for container images, operational safety through automated backups and self-healing,

In [11]:
# ongoing conversations 

user_question_two = "I'm interested in implementing AI with RAG approach, tell me more about how i can do it with elastic as the vector database"

messages.append({"role": "user", "content": user_question_two})

payload = {
    "model": "model",
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

Sure, I'd be happy to explain how you can implement Retrieval-Augmented Generation (RAG) using Elasticsearch as your vector database.

Retrieval-Augented Generation (RAG) is a methodology that combines the power of retrieval systems with generative models. The idea is to retrieve relevant information from a large corpus of data and feed this information into a language model to generate human-like text. Here’s a simplified step-by-step guide on how to implement this using Elasticsearch as your vector database:

1. **Data Preparation**: Firstly, you need to prepare your data. This involves tokenizing your documents and converting them into vectors using techniques like word embeddings (Word2Vec, GloVe), sentence embeddings (Sentence-BERT, Universal Sentence Encoder), or contextual embeddings (ELMo, BERT). These vectors will represent your documents in a high dimensional space where semantically similar texts are close together.

2. **Indexing Documents in Elasticsearch**: Once you've co

<h1>Create Embeddings in Elastic Vector Database</h1>

In [4]:
# install dependencies & imports
!pip install sentence-transformers elasticsearch pandas tqdm

import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm
from elasticsearch.helpers import bulk


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tobiasbueck/helpdesk-github-tickets")

print("Path to dataset files:", path)

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv("cleaned_helpdesk_data.csv", low_memory=False)
print(df.head())

print('\n')

print(df.columns.tolist())

   issue_id  answer_id author              creation_time  \
0    456983          0  almet  2010-12-05T17:31:55+00:00   
1    456983          1  Gui13  2010-12-05T17:43:43+00:00   
2    456983          2  almet  2010-12-05T17:45:28+00:00   
3    456983          3  almet  2010-12-14T15:57:04+00:00   
4    456983          4  Gui13  2010-12-21T21:02:19+00:00   

                                          issue_body  \
0  Hey, Say I add an image in my article, giving ...   
1  Hey, Say I add an image in my article, giving ...   
2  Hey, Say I add an image in my article, giving ...   
3  Hey, Say I add an image in my article, giving ...   
4  Hey, Say I add an image in my article, giving ...   

                                         answer_body  
0  If we do add the SITEURL value, it can break b...  
1  A bit hackish for sure, but I'm not yet good e...  
2  okay, let's go for that so. Will do that when ...  
3  arnaud is currently working on that, see his f...  
4                     Shoul

In [14]:
!pwd

/opt/app-root/src/SEAK_AI


In [6]:
# testing - create a smaller csv
df = pd.read_csv("cleaned_helpdesk_data.csv")
# df_sample = df.head(100)
# df_sample.to_csv("helpdesk_small_sample.csv", index=False)

In [7]:
# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)
print(es.info())

/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.


{'name': 'elasticsearch-sample-es-default-0', 'cluster_name': 'elasticsearch-sample', 'cluster_uuid': 'S_nsm8kqSrye5fgJyJJpLw', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [8]:
# LOGIC
# ideally, all the contents of 'cleaned_body' field from the continous answer_ids grouped under the same issue_id should be in a single vector embedding 

# Load your job dataset
# df = pd.read_csv("helpdesk_small_sample.csv")

# not all values in clean_body are strings, thus throwing TypeError if this line is not runned 
df["answer_body"] = df["answer_body"].fillna("").astype(str)

# Group answers by issue_id and concatenate them in order
grouped = (
    df.sort_values(by=["issue_id", "answer_id"])
      .groupby("issue_id")
      .agg({
          "issue_body": "first",
          "answer_body": lambda x: " ".join(x)
      })
      .reset_index()
)

print(grouped.head(10))
# Embedding Model 
# model = SentenceTransformer("all-MiniLM-L6-v2")  # faced out of memory error 
# model = SentenceTransformer("paraphrase-MiniLM-L3-v2", device="cuda")  # lighter version
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cuda')

# Combine issue_body and answer_body into a single string for embedding
grouped["full_text"] = grouped["issue_body"] + " " + grouped["answer_body"]

# Generate embeddings
grouped["embedding"] = model.encode(grouped["full_text"]).tolist()

# grouped = df.sort_values(by=["issue_id", "answer_id"]).groupby("issue_id")["clean_body"].apply(lambda texts: " ".join(texts)).reset_index()





# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)
print(es.info())

index_name = "helpdesk-embeddings"
if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "issue_id": {"type": "keyword"},
                    "issue_body": {"type": "text"},
                    "answer_body": {"type": "text"},
                    "embedding": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    }
                }
            }
        }
    )

# Prepare docs for bulk indexing
def create_docs(data):
    for _, row in data.iterrows():
        yield {
            "_index": index_name,
            "_id": row["issue_id"],
            "_source": {
                "issue_id": row["issue_id"],
                "issue_body": row["issue_body"],
                "answer_body": row["answer_body"],
                "embedding": row["embedding"]
            }
        }

# Bulk upload
bulk(es, create_docs(grouped))
print("✅ Embeddings stored successfully in Elasticsearch.")

You try to use a model that was created with version 4.1.0, however, your version is 3.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





   issue_id                                         issue_body  \
0    456983  Hey, Say I add an image in my article, giving ...   
1    648603  if you render 5 templates deep, and that templ...   
2    798136  I have an app which is using connect.vhost to ...   
3    814067  Hello, There is a problem with the Pelican's l...   
4    984126  Hi, I'm just about to migrate to pelican and f...   
5    998441  Hi, I'm trying pelican (which is exactly what ...   
6   2349963  The page about **data types** <link> mentions ...   
7   2891909  I'd love support for additional Markdown file ...   
8   3052375  SET in pelican.conf.py: `STATIC_PATHS = ["imag...   
9   3500667  When I create a post using Pelican, images app...   

                                         answer_body  
0  If we do add the SITEURL value, it can break b...  
1  Having a similar issue when rendering partials...  
2  Just so I can find it again (compare view is n...  
3  Having a header like this one in _all_ pelican... 

/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strong

{'name': 'elasticsearch-sample-es-default-0', 'cluster_name': 'elasticsearch-sample', 'cluster_uuid': 'S_nsm8kqSrye5fgJyJJpLw', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made 

✅ Embeddings stored successfully in Elasticsearch.


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.


In [15]:
!nvidia-smi

Wed Apr 23 09:46:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1C.0 Off |                    0 |
| N/A   39C    P0             28W /   70W |     741MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
## for debugging/testing purposes only 

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)

# view available embedding indices 
indices = es.indices.get_alias().keys()
print("Available indices:", list(indices))

# view all stored embeddings 
results = es.search(
    index="helpdesk-embeddings",
    size=10,  # adjust as needed
    _source=["embedding"]  # only return embedding field
)

# view the number of documents in the stored index 
count = es.count(index="helpdesk-embeddings")["count"]
print(f"Num of embeddings in index: {count}")


for doc in results["hits"]["hits"]:
    print(f"ID: {doc['_id']}")
    # print("Embedding:", doc["_source"]["embedding"])

Available indices: ['.security-7', 'helpdesk-embeddings']
Num of embeddings in index: 15784
ID: 456983
ID: 648603
ID: 798136
ID: 814067
ID: 984126
ID: 998441
ID: 2349963
ID: 2891909
ID: 3052375
ID: 3500667


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.
/tmp/ipykernel_67/250962508.py:11: ElasticsearchWarning: this request accesses system indices: [.security-7], but in a future major version, direct access to system indices will be prevented by default
  indices = es.indices.get_alias().keys()
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/

In [11]:
# delete the entire index after checking 
# ONLY RUN TO DELETE THE ENTIRE INDEX CREATED - SO THAT IT IS A CLEAN STATE IN VECTOR DB

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)

# es.indices.delete(index="helpdesk-embeddings", ignore_unavailable=True)
# print("Index deleted.\n")

# view available embedding indices 
indices = es.indices.get_alias().keys()
print("Available indices:", list(indices))

Available indices: ['.security-7', 'helpdesk-embeddings']


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.
/tmp/ipykernel_67/1093246816.py:15: ElasticsearchWarning: this request accesses system indices: [.security-7], but in a future major version, direct access to system indices will be prevented by default
  indices = es.indices.get_alias().keys()


<h1>Retrieving Top 3 Most Relevant Embeddings from Vector DB</h1>

In [13]:
# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)

mapping = es.indices.get_mapping(index="helpdesk-embeddings")
print(mapping["helpdesk-embeddings"]["mappings"]["properties"].get("embedding", "Field not found"))

{'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  # Force connect early to allow us to validate the connection.


In [14]:
import warnings
import urllib3
from urllib3.exceptions import InsecureRequestWarning

# Disable SSL warnings
warnings.simplefilter('ignore', InsecureRequestWarning)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# elasticsearch client 
es = Elasticsearch(
    hosts=["https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com"], 
    basic_auth=('myadmin', 'newpassword123'), 
    verify_certs=False
)

model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device='cpu')


# get top 3 most relevant embeddings 
def search_embeddings(query, top_n=3):
    # Step 1: Embed the query using the same model
    query_embedding = model.encode(query).tolist()

    # Step 2: Search the Elasticsearch index for the closest vector matches
    response = es.search(
        index="helpdesk-embeddings",
        body={
            "size": top_n,
            "query": {
                "knn": {
                    "field": "embedding",
                    "k": top_n,
                    "num_candidates": 100,
                    "query_vector": query_embedding
                }
            }
        }
    )

    hits = response["hits"]["hits"]
    relevant_answers = []

    for hit in hits:
        issue_id = hit["_source"]["issue_id"]
        answer_body = hit["_source"]["answer_body"]
        score = hit["_score"]
        relevant_answers.append({
            "issue_id": issue_id,
            "answer_body": answer_body,
            "score": score
        })

    return relevant_answers

user_query = "I have an issue with Pelican license, where pelican doe not have a parser.py"
top_matches = search_embeddings(user_query) 

for match in top_matches:
    print(f"Issue ID: {match['issue_id']}")
    print(f"Answer Body: {match['answer_body']}")
    print(f"Score: {match['score']}")
    print("="*50)
    

/opt/app-root/lib64/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://elasticsearch-sample-demo-chatbot.apps.cluster-c5xdq.c5xdq.sandbox1264.opentlc.com:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
You try to use a model that was created with version 4.1.0, however, your version is 3.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Issue ID: 814067
Answer Body: Having a header like this one in _all_ pelican's files seems more than annoying to me. After some search, it seems that the recommended practice is to to add the notice in every file, but not doing so will not invalidate the license or change the meaning of it (<link> Anyway, I will change the LICENSE text. > After some search, it seems that the recommended practice is to to add the notice in every file, but not doing so will not invalidate the license Interesting, I did not know.
Score: 0.79530144
Issue ID: 1267992009
Answer Body: `pelican` doesn't have a `parser.py`. This is likely from a plugin. Try removing plugins or running `pelican` with `--debug` to see which file/plugin is the issue. ![image](<link> I don't recall installing this plugin. I'm currently in dependency hell, can't get linker plugin to import a module from itself. Hey David. Were you eventually able to get everything sorted out?
Score: 0.7677038
Issue ID: 683970143
Answer Body: Hi Mich

/tmp/ipykernel_67/2543801249.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


<h1>Model Inference - Prompt Construction</h1>

In [31]:
infer_endpoint = "http://model-predictor.minio.svc.cluster.local:8080" #Change infer endpoint here
infer_url = f"{infer_endpoint}/v1/chat/completions"

user_query = "I have the error BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]') when running elasticsearch"
# Build the retrieved context
context = "\n\n---\n\n".join([f"Issue ID: {match['issue_id']}\nAnswer: {match['answer_body']}" for match in top_matches])

# Step 2: Construct messages for /v1/chat/completions (include chain-of-thought reasoning model)
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful and knowledgeable support assistant. "
            "Answer the user's question clearly and accurately. "
            "Use the provided context from our database to support your response wherever relevant. "
            "Think step by step when forming your answer, reasoning through each part of the problem before responding. "
            "If the context does not contain enough information to answer the question, "
            "politely let the user know and suggest alternative next steps if appropriate.\n\n"
            f"Context:\n{context}"
        )
    },
    {
        "role": "user",
        "content": user_query
    }
]



payload = {
    "model": "model",
    "messages": messages,
    "max_tokens": 1024,
    "temperature": 0.3, # Controls randomness, lower temp for factuality 
    "top_p": 1,
    "n": 1, # Number of completions to generate
    "repetition_penalty": 1.1, # Penalize repeated tokens (1 = no penalty)
    "presence_penalty": 0.2, # Discourage mentioning same concepts again
    "frequency_penalty": 0.2, # Discourage repeating the *same words* too frequently
    "stream": False # If True, stream tokens back (like a live typewriter)
}

# query LLM server
response = requests.post(infer_url, json=payload)

# prints the whole response json
# print(response.json())

output_body = response.json()
generated_response = output_body['choices'][0]['message']['content']
print(generated_response.strip())

# add to the messages list 
messages.append({"role": "system", "content": generated_response.strip()})

I'm sorry for the confusion, but the provided context doesn't include any information related to Elasticsearch errors. However, I can still try to help you troubleshoot this issue based on common practices.

The error message `BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]')` typically occurs when there's an issue with the HTTP headers sent by your client (in this case, presumably your Elasticsearch script or application). 

Here are a few things you could check:

1. **Check Accept Header**: The `Accept` header specifies what content types the client understands. Ensure that it's set correctly. For example, if you're expecting JSON responses, it should look something like this: `Accept: application/json`.

2. **Check Content-Type Header**: The `Content-Type` header tells Elasticsearch what format the data being sent is in. If you're sending data in JSON format, it should be set to `application/json`.

3. 

<h1>Request Function</h1>

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [ ]:
import requests


def rest_request(prompt):
    json_data = {
        "model": "llm",
        "prompt": [
            prompt
        ],
        "max_tokens": 512,
        "temperature": 1,
        "top_p": 1,
        "n": 1,
        "stream": False,
        "logprobs": 0,
        "echo": False,
        "stop": [
            "string"
        ],
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "best_of": 1,
        "user": "string",
        "top_k": -1,
        "ignore_eos": False,
        "use_beam_search": False,
        "stop_token_ids": [
            0
        ],
        "skip_special_tokens": True,
        "spaces_between_special_tokens": True,
        "repetition_penalty": 1,
        "min_p": 0,
        "include_stop_str_in_output": False,
        "length_penalty": 1
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    return response.json()

In [ ]:
prediction = rest_request("What accelerators are supported in openshift AI?")
prediction

In [56]:
!curl http://model-predictor.minio.svc.cluster.local:8080/health

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
